In [1]:
import bw2data as bd
import bw2calc as bc
import bw2io as bi

import numpy as np
import json, os

#Local files
from import_databases import *

In [2]:
with open('global_settings.json', 'rb') as f:
    settings = json.load(f)
which_pc = settings['which_pc']
which_pc

'local'

### TODO -> give paths to databases files

In [3]:
if which_pc == 'local':
    # Older databases
    ei33_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.3 cutoff/datasets'  
    ex22_path = '/Users/akim/Documents/LCA_files/exiobase_2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ag12_path = '/Users/akim/Documents/LCA_files/agribalyse 1.2/AGRIBALYSEv1.2_Complete.xml'
    ei36_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.6 cutoff/datasets'  
    # Newer databases
    ei371_path = '/Users/akim/Documents/LCA_files/ecoinvent 3.7.1 cutoff/datasets'  
    ex381_path = '/Users/akim/Documents/LCA_files/exiobase_3.8.1_monetary/IOT_2015_pxp/'
    ex381_name = "Exiobase 3.8.1 Monetary 2015"
    sut_path = '/Users/akim/Documents/LCA_files/exiobase_SUT/data/'
    ag13_path = '/Users/akim/Documents/LCA_files/agribalyse 1.3/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/Users/akim/Documents/LCA_files/HABE_2017/'
    co_path = 'data/es8b01452_si_002.xlsx'
elif which_pc == 'merlin':
    ex22_path = '/data/user/kim_a/LCA_files/exiobase_22/mrIOT_IxI_fpa_coefficient_version2.2.2'
    ei33_path = '/data/user/kim_a/LCA_files/ecoinvent_33_cutoff/datasets'  
    ei36_path = '/data/user/kim_a/LCA_files/ecoinvent_36_cutoff/datasets'  
    ag13_path = '/data/user/kim_a/LCA_files/agribalyse_13/Agribalyse CSV FINAL_no links_Nov2016v3.CSV'
    habe_path = '/data/user/kim_a/LCA_files/HABE_2017'
    co_path = 'data/es8b01452_si_002.xlsx'

# TODO -> choose project

In [4]:
project = "GSA for protocol"
bd.projects.set_current(project)

ei36_name = 'ecoinvent 3.6 cutoff'
ei371_name = 'ecoinvent 3.7.1 cutoff'
ei_option = "371"
if ei_option == "36":
    ei_name = ei36_name
    ei_path = ei36_path
elif ei_option == "371":
    ei_name = ei371_name
    ei_path = ei371_path
    
ex_name = ex381_name
ex_path = ex381_path

In [ ]:
# bd.projects.delete_project(project, True)

In [ ]:
try:
#     del bw.databases["Agribalyse 1.3 - {} - new biosphere".format(ei_name)]
#     del bw.databases["Agribalyse 1.3 - {}".format(ei_name)]
    del bd.databases["CH consumption 1.0"]
except:
    pass

In [8]:
bd.databases

Databases dictionary with 3 object(s):
	CH consumption 1.0
	biosphere3
	ecoinvent 3.7.1 cutoff

### Import necessary databases

Import of Exiobase 3 might need latest brightway 2.5 version. We recommend creating new conda environment. Until there is an official release, following packages:

- brightway2-io
- brightway2-data
- brightway2-calc
- matrix_utils
- bw_processing

1. cannot be conda or pip install
2. need to be git cloned from the respective websites, search here: https://github.com/brightway-lca
3. once cloned, install each of the packages with `pip install -e .` (editable install) in their directories, where setup.py is located
4. these packages are under active development, so make sure to pull the newest changes from time to time

In [6]:
co_name = CONSUMPTION_DB_NAME

if project == "GSA for protocol":
#     bw-migrations             0.1                      pypi_0    pypi
#     bw2analyzer               0.10                     pypi_0    pypi
#     bw2calc                   1.8.0                    pypi_0    pypi
#     bw2data                   3.6.2                    pypi_0    pypi
#     bw2io                     0.8.3.1                  pypi_0    pypi
#     bw2parameters             0.6.6                    pypi_0    pypi
    bi.bw2setup()
    import_ecoinvent(ei_path, ei_name)
    create_ecoinvent_33_project(ei33_path)
#     import_exiobase_3(ex_path, ex_name)
    exclude_dbs = [
        'heia', 
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei_name),
        'exiobase 2.2'
    ]
    co = import_consumption_db(
        co_path, 
        habe_path, 
        exclude_dbs=exclude_dbs, 
        ei_name=ei_name, 
        ex_path=ex_path,
        sut_path=sut_path,
    )
    add_consumption_activities(co_name, habe_path, habe_year='151617', option='aggregated',)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)
    
elif project == "GSA for protocol_ex":
    bi.bw2setup()
    import_ecoinvent(ei_path, ei_name)
    create_ecoinvent_33_project(ei33_path)
    import_exiobase_3(ex_path, ex_name)
    exclude_dbs = [
        'heia', 
        'Agribalyse 1.2',
        'Agribalyse 1.3 - {}'.format(ei_name),
    ]
    co = import_consumption_db(
        co_path, 
        habe_path, 
        exclude_dbs=exclude_dbs, 
        ei_name=ei_name, 
        ex_path=ex_path,
        sut_path=sut_path,
    )
    add_consumption_activities(co_name, habe_path, habe_year='151617', option='aggregated',)
    add_consumption_categories(co_name, co_path)
    add_consumption_sectors(co_name)

Biosphere database already present!!! No setup is needed
ecoinvent 3.7.1 cutoff database already present!!! No import is needed
Biosphere database already present!!! No setup is needed
ecoinvent 3.3 cutoff database already present!!! No import is needed
--> Creating consumption_db.xlsx
Extracted 1 worksheets in 5.28 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: csv_add_missing_exchanges_section
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertain

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 06/02/2021 12:36:58
  Finished: 06/02/2021 12:36:59
  Total time elapsed: 00:00:00
  CPU %: 107.00
  Memory %: 1.97
Created database: CH consumption 1.0
['ch hh all consumption aggregated' (1 month of consumption, CH, None)]


In [ ]:
# %%time 

# co_name = CONSUMPTION_DB_NAME

# if project == 'GSA for ecoinvent':
#     bw.bw2setup()
#     import_exiobase_22(ex22_path)
#     import_ecoinvent(ei_path, ei_name)
#     import_agribalyse_13(ag13_path, ei_name)
#     # Consumption DB
#     create_ecoinvent_33_project(ei33_path)
#     import_consumption_db(co_path, habe_path, exclude_dbs=['heia'])
#     add_consumption_activities(co_name, habe_path)

# elif project == 'GSA for paper':
#     bw.bw2setup()
#     import_ecoinvent(ei_path, ei_name)
#     # Consumption DB
#     create_ecoinvent_33_project(ei33_path)
#     exclude_dbs = [
#         'heia', 
#         'EXIOBASE 2.2',
#         'Agribalyse 1.2',
#         'Agribalyse 1.3 - {}'.format(ei_name),
#     ]
#     import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
#     add_consumption_activities(co_name, habe_path)
#     add_consumption_categories(co_name, co_path)
#     add_consumption_sectors(co_name)
    
# elif project == 'GSA no exiobase':
#     bw.bw2setup()
#     import_ecoinvent(ei_path, ei_name)
#     import_agribalyse_13(ag13_path, ei_name)
#     # Consumption DB
#     create_ecoinvent_33_project(ei33_path)
#     exclude_dbs = [
#         'heia', 
#         'EXIOBASE 2.2',
#     ]
#     import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
#     add_consumption_activities(co_name, habe_path)
    
# elif project == 'GSA for setac':
#     bw.bw2setup()
#     import_ecoinvent(ei_path, ei_name)
#     # Consumption DB
#     create_ecoinvent_33_project(ei33_path)
#     exclude_dbs = [
#         'heia', 
#         'EXIOBASE 2.2',
#         'Agribalyse 1.2',
#         'Agribalyse 1.3 - {}'.format(ei_name),
#     ]
#     import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs)
#     add_consumption_activities(co_name, habe_path)
#     add_consumption_categories(co_name, co_path)
#     add_consumption_sectors(co_name)
    
# elif 'GSA for oases' in project:
#     bw.bw2setup()
#     import_ecoinvent(ei_path, ei_name)
#     # Consumption DB
#     create_ecoinvent_33_project(ei33_path)
#     import_agribalyse_13(ag13_path, ei_name)
#     exclude_dbs = [
#         'heia', 
#         'EXIOBASE 2.2',
#     ]
#     import_consumption_db(co_path, habe_path, exclude_dbs=exclude_dbs, ei_name=ei_name)
#     add_consumption_activities(co_name, habe_path)
#     add_consumption_categories(co_name, co_path)
#     add_consumption_sectors(co_name)
    
# elif "GSA for protocol" in project:
#     bw.bw2setup()
#     import_ecoinvent(ei_path, ei_name)
#     create_ecoinvent_33_project(ei33_path)
#     import_exiobase_3(ex_path, ex_name)
#     exclude_dbs = [
#         'heia', 
#         'Agribalyse 1.2',
#         'Agribalyse 1.3 - {}'.format(ei_name),
#     ]
#     co = import_consumption_db(
#         co_path, 
#         habe_path, 
#         exclude_dbs=exclude_dbs, 
#         ei_name=ei_name, 
#         ex_path=ex_path,
#         sut_path=sut_path,
#     )
#     add_consumption_activities(co_name, habe_path, habe_year='151617', option='aggregated',)
#     add_consumption_categories(co_name, co_path)
#     add_consumption_sectors(co_name)

In [ ]:
co = bd.Database('CH consumption 1.0')
for act in co:
    excs = [exc.input for exc in act.exchanges()]
    rep_inputs = {exc for exc in excs if excs.count(exc) > 1}
    if len(rep_inputs)>0:
        print(act, rep_inputs)     

In [ ]:
demand, data_objs, remapping_dicts = bd.prepare_lca_inputs(demand)

In [ ]:
lca= bc.LCA(demand=demand, data_objs=data_objs, remapping_dicts=remapping_dicts)

In [7]:
%%time
co = bd.Database('CH consumption 1.0')
# demand_act = co.search('average consumption aggregated')[0]
demand_act = co.search('food sector')[0]
demand = {demand_act: 1}
method = ('IPCC 2013', 'climate change', 'GWP 100a')
print(demand_act)

lca = bc.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

'Food and non-alcoholic beverages sector' (1 month of consumption, CH, None)
CPU times: user 11.5 s, sys: 2.07 s, total: 13.5 s
Wall time: 3 s


211.57670115335657

# Contribution of food items

In [ ]:
%%time
fus = [{exc.input: exc.amount} for exc in demand_act.exchanges()]
scores = {}
for fu in fus:
    lca = bw.LCA(fu, method)
    lca.lci()
    lca.lcia()
    scores[list(fu.keys())[0]['name']] = {
        "score": lca.score,
    }
    print(fu, lca.score)

# Contribution of sectors

In [ ]:
%%time
fus = [act for act in co if 'sector' in act['name']]
scores = {}
for fu in fus:
    excs = {exc.input['name']: "{} {}".format(exc['amount'], exc.input['unit']) for exc in fu.exchanges()
           if exc['type']=='technosphere'}
    lca = bw.LCA({fu: 1}, method)
    lca.lci()
    lca.lcia()
    scores[fu['name']] = {
        "exchanges": excs,
        "score": lca.score,
    }
    print(fu, lca.score)

In [ ]:
write_dir = Path('write_files')
filepath = write_dir / "contribution_scores_sectors.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(scores, f)

# Scores for all consumption activities

In [ ]:
import brightway2 as bw
import pickle
from pathlib import Path

In [ ]:
%%time
write_dir = Path("write_files")

project = 'GSA for protocol'
bw.projects.set_current(project)
co = bw.Database("CH consumption 1.0")
method = ('IPCC 2013', 'climate change', 'GWP 100a')
contribution_scores = {}
i = 0
for act in list(co):
    demand = {act: 1}
    lca = bw.LCA(demand, method)
    lca.lci()
    lca.lcia()
    act_name = act["name"]
    contribution_scores[act_name] = lca.score
    print(i, lca.score, act_name)
    i+=1

In [ ]:
filepath = write_dir / "contribution_scores.pickle"
with open(filepath, 'wb') as f:
    pickle.dump(contribution_scores, f)

# Contribution of sectors, where each fu within sector has amount = 1

In [ ]:
# %%time
# fus = [act for act in co if 'sector' in act['name']]
# scores = {}
# for fu in fus:
#     new_fu = {exc.input: 1 for exc in fu.exchanges() if exc['type']=='technosphere'}
#     new_fu_str = {exc.input['name']: "1 {}".format(exc.input['unit']) 
#                   for exc in fu.exchanges() if exc['type']=='technosphere'}
#     lca = bw.LCA(new_fu, method)
#     lca.lci()
#     lca.lcia()
#     scores[fu['name']] = {
#         "exchanges": new_fu_str,
#         "score": lca.score,
#     }
#     print(fu, lca.score)

In [ ]:
# path_scores = 'write_files/contribution_scores_sectors_allfu1.pickle'
# with open(path_scores, 'wb') as f:
#     pickle.dump(scores, f)

# Contribution of specific sectors

In [ ]:
# demand_act = co.search('ch hh average consumption')
# assert len(demand_act) == 1
# demand_act = demand_act[0]

# purchase, transportation_services, transport_by_air, recreation, package_holidays = {}, {}, {}, {}, {}
# purchase_name = 'Purchase and operation of vehicles'
# transportation_services_name = "Transportation services"
# transport_by_air_name = "Passenger transport by air"
# recreation_name = "Recreation and culture"
# package_holidays_name = "Package holidays"
# for exc in demand_act.exchanges():
#     if exc.input.get('category_middle') == purchase_name:
#         purchase.update({
#             exc.input: exc.amount,
#         })
#     if exc.input.get('category_middle') == transportation_services_name:
#         if exc.input.get('category_fine') == transport_by_air_name:
#             transport_by_air.update({
#                 exc.input: exc.amount,
#             })
#         else:
#             transportation_services.update({
#                 exc.input: exc.amount,
#             })
#     if exc.input.get('category_coarse') == recreation_name:
#         if exc.input.get('category_middle') == package_holidays_name:
#             package_holidays.update({
#                 exc.input: exc.amount,
#             })
#         else:
#             recreation.update({
#                 exc.input: exc.amount,
#             })
# fus = {
#     purchase_name: purchase,
#     transportation_services_name: transportation_services,
#     transport_by_air_name: transport_by_air,
#     recreation_name: recreation,
#     package_holidays_name: package_holidays,
# } 

In [ ]:
# contribution_scores = {}
# for name,demand in fus.items():
#     new_fu = {k:1 for k,v in demand.items()}
#     new_fu_str = {k['name']: "1 {}".format(k['unit']) for k,v in demand.items()}
#     lca = bw.LCA(new_fu, method)
#     lca.lci()
#     lca.lcia()
#     contribution_scores[name] = {
#         "LCA score": lca.score,
#         "Exchanges": new_fu_str,
#     }
#     print(lca.score, name)

In [ ]:
# path_scores = 'write_files/contribution_scores_5categories_allfu1.pickle'
# with open(path_scores, 'wb') as f:
#     pickle.dump(contribution_scores, f)

In [ ]:
# from bw2calc import MonteCarloLCA
# mc = MonteCarloLCA(demand, method)
# [next(mc) for _ in range(5)]

# Comparison between final demand of exiobase 2.2 and 3.8.1

In [ ]:
import brightway2 as bw
import numpy as np
import json, os, pickle
import re

#Local files
from import_databases import *

In [ ]:
ex22_path = '/Users/akim/Documents/LCA_files/exiobase_2.2/mrIOT_IxI_fpa_coefficient_version2.2.2'
ex381_path = '/Users/akim/Documents/LCA_files/exiobase_3.8.1_monetary/IOT_2015_pxp/'

filename22 = "mrFinalDemand_version2.2.2.txt"
columns22 = ['Unnamed: 0', 'Unnamed: 1', 'CH']
filename381 = "Y.txt"
columns381 = ['region', 'Unnamed: 1', 'CH']
filepath22 = Path(ex22_path) / filename22
filepath381 = Path(ex381_path) / filename381

df2 = pd.read_table(filepath22)
df2 = df2[columns22]
df2.columns = ['location', 'name', 'hh_consumption_2007_euro']
df2 = df2.dropna()
df2.index = np.arange(len(df2))

df3 = pd.read_table(filepath381)
df3 = df3[columns381]
df3.columns = ['location', 'name', 'hh_consumption_2015_euro']
df3 = df3.dropna()
df3.index = np.arange(len(df3))

# Harmonize names in exiobase 2 and 3, use names from exiobasee 3 everywhere
filepath_ex381_names = 'data/migrations/exiobase-3.8.1.json'
with open(filepath_ex381_names, 'r') as f:
    ex_names_dict_raw = json.load(f)
ex_names_dict = {}
for el in ex_names_dict_raw['data']:
    ex_names_dict[el[0][0]] = el[1]['name']

for i,row in df2.iterrows():
    new_name = re.sub(r'\(.[0-9]+\)', '', row['name']).rstrip()
    if 'Manufacture of' in new_name:
        new_name = new_name[15:].capitalize()
    row['name']=ex_names_dict.get(new_name, new_name)   
for i,row in df3.iterrows():
    new_name = re.sub(r'\(.[0-9]+\)', '', row['name']).rstrip()
    row['name']=new_name

filepath_acts = 'write_files/exiobase_in_consumption_db.pickle'
with open(filepath_acts, 'rb') as f:
    ex_acts = pickle.load(f)
    
chf_to_euro_2007 = 0.594290
chf_to_euro_2015 = 0.937234

frames2, frames3 = [], []
for act in ex_acts:
    frame2 = df2[df2['name'] == act]
    if len(frame2)==0:
        print('{} not found in ex2'.format(act))
    else:
        sum2 = sum([float(val) for val in frame2['hh_consumption_2007_euro'].values])
        frame2 = frame2.append(
            {'location': 'SUM_ALL', 'name': act, 'hh_consumption_2007_euro': sum2}, ignore_index=True
        )
        frames2.append(frame2)
    
    frame3 = df3[df3['name'] == act]
    if len(frame3)==0:
        print('{} not found in ex3'.format(act))
    else:
        sum3 = sum([float(val) for val in frame3['hh_consumption_2015_euro'].values])
        frame3 = frame3.append(
            {'location': 'SUM_ALL', 'name': act, 'hh_consumption_2015_euro': sum3}, ignore_index=True
        )
        frames3.append(frame3) 
    
df2_subset = pd.concat(frames2)
df3_subset = pd.concat(frames3)
acts2 = set(df2_subset['name'])
acts3 = set(df3_subset['name'])

df2_subset['hh_consumption_2007_euro'] = [float(val) for val in df2_subset['hh_consumption_2007_euro'].values]
df3_subset['hh_consumption_2015_euro'] = [float(val) for val in df3_subset['hh_consumption_2015_euro'].values]

df = df3_subset.merge(df2_subset, how='left')
df['hh_consumption_2015_chf'] = df['hh_consumption_2015_euro'].values/chf_to_euro_2015
df['hh_consumption_2007_chf'] = df['hh_consumption_2007_euro'].values/chf_to_euro_2007
df['ratio_2015_to_2007_chf'] = df['hh_consumption_2015_chf'] / df['hh_consumption_2007_chf']
df['ratio_2007_to_2015_chf'] = df['hh_consumption_2007_chf'] / df['hh_consumption_2015_chf']

In [ ]:
df.to_excel('write_files/final_demand_comparison_2007_2015.xlsx')

# Comparison of contribution scores: 
## linking of consumption database to i) exiobase_industry_workaround vs ii) exiobase 3.8.1

In [ ]:
import brightway2 as bw
import pickle
from pathlib import Path
import pandas as pd

In [ ]:
bw.projects.set_current("GSA for protocol")
co = bw.Database("CH consumption 1.0")

In [ ]:
fp_scores_sectors = 'write_files/contribution_scores_sectors.pickle'
fp_scores_sectors_old = 'write_files/contribution_scores_sectors_exworkaround.pickle'
fp_scores = 'write_files/contribution_scores.pickle'
fp_scores_old = 'write_files/contribution_scores_exworkaround.pickle'
fp_exiobase_old = 'write_files/exiobase_lca_exworkaround.pickle'
# with open(fp_scores_sectors, 'rb') as f:
#     scores_sectors = pickle.load(f)
# with open(fp_scores_sectors_old, 'rb') as f:
#     scores_sectors_old = pickle.load(f)
with open(fp_scores, 'rb') as f:
    scores = pickle.load(f)
with open(fp_scores_old, 'rb') as f:
    scores_old = pickle.load(f)
with open(fp_exiobase_old, 'rb') as f:
    scores_ex22 = pickle.load(f)
    
df = pd.DataFrame.from_dict({'scores': scores, 'scores_old': scores_old})
df['diff'] = df['scores'] - df['scores_old']
df['diff_abs'] = abs(df['scores'] - df['scores_old'])
df['ratio'] = df['scores'] / df['scores_old']

bw.projects.set_current('GSA for protocol')
co = bw.Database('CH consumption 1.0')
ex = bw.Database('Exiobase 3.8.1 Monetary 2015')
consumption = [act for act in co if 'average consumption' in act['name']][0]

amounts = {}
units = {}
for exc in consumption.exchanges():
    amounts[exc.input['name']] = exc.amount
    units[exc.input['name']] = exc.input['unit']
    
for i,row in df.iterrows():
    df.at[i,'amount from fu'] = amounts.get(row.name)
    df.at[i,'unit from fu'] = units.get(row.name)

In [ ]:
df.to_excel('write_files/contribution_scores_comparison_exworkaround_ex3.xlsx')

In [ ]:
inland_water_transport = [act for act in ex if 'Inland water transport' in act['name'] and 'PT' in act['location']][0]
# LCA score is 2 times higher now
education = [act for act in ex if "Education" in act['name'] and 'CN' in act['location']][0]
# LCA score is 1.4 times higher now
wearing_apparel = [act for act in ex if "Wearing apparel" in act['name'] and 'NO' in act['location']][0]
# ???
tobacco = [act for act in ex if "Tobacco" in act['name'] and 'ID' in act['location']][0]
leather = [act for act in ex if "leather" in act['name'].lower() and 'BG' in act['location']][0]

other_land_transport = [act for act in ex if "Other land transport" in act['name'] and 'RU' in act['location']][0]
# LCA score is 0.6 times lower now
fabricated_metal = [act for act in ex if "Fabricated metal" in act['name'] and 'TR' in act['location']][0]
# LCA score is 0.3 times lower now
post_telecom = [act for act in ex if "Post and tele" in act['name'] and 'SI' in act['location']][0]
# LCA score is 0.4 times lower now
recreation_culture_sport = [act for act in ex if "Recreational, cul" in act['name'] and 'ID' in act['location']][0]
# LCA score is 0.4 times lower now


In [ ]:
demand = {leather: 1}
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
lca.score /1176234.1432328927

In [ ]:
scores_ex22

In [ ]:
list(consumption.exchanges())

In [ ]:
bw.projects.set_current('GSA for protocol')
co = bw.Database('CH consumption 1.0')
ex = bw.Database('Exiobase 3.8.1 Monetary 2015')
consumption = [act for act in co if 'average consumption' in act['name']][0]

recreation = [act for act in co if 'Recreation' in act['name']][0]
air = [act for act in ex if 'Air transport services' in act['name'] and 'CH' in act['location']][0]
dental = [act for act in co if 'Dental care' in act['name']][0]
personal_care = [act for act in co if 'Other products for personal care' in act['name']][0]
medicines = [act for act in co if 'Medicines' in act['name']][0]
cutlery = [act for act in co if 'Cutlery' in act['name']][0]
internet = [act for act in co if "Services of internet providers" in act['name']][0]
taxi = [act for act in co if "Taxi" in act['name']][0]
education = [act for act in co if "Education services" in act['name']][0]
telefax = [act for act in co if "Purchases and rent of telephones and telefax" in act['name']][0]
radio = [act for act in ex if "Radio, television and communication equipment and" in act['name'] and
        act['location']=='TR'][0]
post = [act for act in ex if "Post and telecommunication services" in act['name'] and
        act['location']=='SE'][0]
furniture = [act for act in ex if "Furniture; other manufactured goods" in act['name'] and
        act['location']=='FI'][0]
motor = [act for act in ex if "Motor vehicles, trailers and semi-trailers" in act['name'] and
        act['location']=='GR'][0]
other_transport = [act for act in ex if "Other transport equipment" in act['name'] and
        act['location']=='CN'][0]

furnishings = [act for act in co if "Furnishings" in act['name']][0] 
# funishings contains activities whose impact was reduced due to lower shares from the CH final demand from exiobase,
# eg cutlery, kitchen utensils and others that contain "Fabricated metal products, except machinery and equipment"
misc = [act for act in co if "Miscellaneous goods and services sector" in act['name']][0]
# same goes for misc, but here hair, dental care products, jewellery, etc have chemicals that now have 0 shares
health = [act for act in co if "Health" in act['name']][0]
# health is probably lower due to medicines which also contain chemicals
communication = [act for act in co if "Communication" in act['name']][0]
# Impact of Radio, television and communication equipment and apparatus is >2 times smaller
# Impact of Post and telecommunication is 3.7 times smaller
durable_recreation = [act for act in co if 'Durable goods for recreation' in act['name']][0]
# Impact of furniture is ca 2 times less compared to exiobase 2.2, but aggregated share is now 4.5 times more
# Impact of motor vehicles is ca 1.33 times less, but aggregated share is now 2.6 times more
# Impact of other transport equipment is 2.5 times less, but aggregated share is now 32 times more


In [ ]:
demand = {consumption: 1}
lca = bw.LCA(demand, method)
lca.lci()
lca.lcia()
lca.score

In [ ]:
s = 0
for act in furnishings.exchanges():
    print("{} {:10}, {}".format(act.amount, act.input['name'], act.input['location']))
    if 'Other transport equipment' in act.input['name']:
        s += act.amount

In [ ]:
import brightway2 as bw
import pickle
from pathlib import Path
import pandas as pd

bw.projects.set_current('GSA for protocol')
co = bw.Database('CH consumption 1.0')
ex = bw.Database('Exiobase 3.8.1 Monetary 2015')
consumption = [act for act in co if 'average consumption' in act['name']][0]
ipcc_tuple = ('IPCC 2013', 'climate change', 'GWP 100a')
rcpe_tuple = ('ReCiPe Midpoint (H) V1.13', 'climate change', 'GWP100')

In [ ]:
ipcc = bw.Method(ipcc_tuple)
ipcc_data = {}
for m in ipcc.load(): 
    act = bw.get_activity(m[0])
    act_key = "{} -- {}".format(act['name'], act['categories'])
    ipcc_data[act_key] = m[1]
#     print("{:5.2f} {:10s} {:20s} {}".format(m[1], act['unit'], act['name'], act['categories']))

In [ ]:
rcpe = bw.Method(rcpe_tuple)
rcpe_data = {}
for m in rcpe.load(): 
    act = bw.get_activity(m[0])
    act_key = "{} -- {}".format(act['name'], act['categories'])
    rcpe_data[act_key] = m[1]
#     print("{:5.2f} {:10s} {:20s} {}".format(m[1], act['unit'], act['name'], act['categories']))

In [ ]:
df = pd.DataFrame.from_dict({'ipcc': ipcc_data, "rcpe": rcpe_data})
df['diff'] = df['ipcc'] - df['rcpe']

In [ ]:
df.to_excel('a.xlsx')

In [ ]:
len(rcpe_data)

In [ ]:
lca.score

In [ ]:
demand = {consumption: 1}
lca = bw.LCA(demand, rcpe_tuple)
lca.lci()
lca.lcia()
lca.score

In [ ]:
ch_params = lca.characterization_mm.groups[0]
pos = ch_params.package.data[0]
val = ch_params.package.data[1]
nch = len(pos)
for i in range(nch):
    break